In [52]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import scipy as sp
import time
from datetime import datetime, date, timedelta

from dateutil.relativedelta import relativedelta

import io
from io import StringIO 
#import datedelta
import calendar
import  csv
import json
import random

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats
def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['minute_col'] = 0
        df['hour_col'] = 0
        new_col = df.columns.get_loc('minute_col')
        new_col2 = df.columns.get_loc('hour_col')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
            arr[i,new_col2] = pd.Timestamp(arr[i,time_col]).hour
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    df = df.drop_duplicates(subset=['utc_timestamp'])
    print(df.shape)
    return df

In [14]:
df2

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col
0,2016-01-03T22:00:00.000000000Z,7404,0.68374,0.68374,0.67678,0.67683,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00691,0.00696,0.0,0.00005,0.00691,0
1,2016-01-04T02:00:00.000000000Z,5726,0.67688,0.677,0.67132,0.67168,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,-0.0052,0.00568,0.00012,0.00036,0.0052,0
2,2016-01-04T06:00:00.000000000Z,8797,0.67166,0.6739,0.67166,0.67246,2016-01-04 06:00:00+00:00,2016-01-04 01:00:00-05:00,0.0008,0.00224,0.00144,0.0,0.0008,0
3,2016-01-04T10:00:00.000000000Z,7013,0.67246,0.67707,0.67208,0.67578,2016-01-04 10:00:00+00:00,2016-01-04 05:00:00-05:00,0.00332,0.00499,0.00129,0.00038,0.00332,0
4,2016-01-04T14:00:00.000000000Z,9421,0.6758,0.67858,0.67514,0.67678,2016-01-04 14:00:00+00:00,2016-01-04 09:00:00-05:00,0.00098,0.00344,0.0018,0.00066,0.00098,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10667,2022-11-04T01:00:00.000000000Z,9621,0.58438,0.5872,0.58438,0.5869,2022-11-04 01:00:00+00:00,2022-11-03 21:00:00-04:00,0.00252,0.00282,0.0003,0.0,0.00252,0
10668,2022-11-04T05:00:00.000000000Z,19838,0.58694,0.5886,0.58632,0.5884,2022-11-04 05:00:00+00:00,2022-11-04 01:00:00-04:00,0.00146,0.00228,0.0002,0.00062,0.00146,0
10669,2022-11-04T09:00:00.000000000Z,26412,0.58842,0.58964,0.58586,0.58872,2022-11-04 09:00:00+00:00,2022-11-04 05:00:00-04:00,0.0003,0.00378,0.00092,0.00256,0.0003,0
10670,2022-11-04T13:00:00.000000000Z,40303,0.58877,0.58912,0.58676,0.58849,2022-11-04 13:00:00+00:00,2022-11-04 09:00:00-04:00,-0.00028,0.00236,0.00035,0.00173,0.00028,0


In [28]:
df = df.merge(df2[['utc_timestamp','delta']].rename(columns = {'utc_timestamp':'utc_timestamp','delta':'delta4'}),on = 'utc_timestamp',how = 'left')
df['delta4'] = df['delta4'].fillna(-100)

(490624, 27) (10664, 14) (490624, 28)


In [29]:
df3.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col,support_indicator,delta4
0,2016-01-03 22:00:00,10,0.68374,0.68374,0.6833,0.68346,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,0.68346,0.68346,...,0,0,-0.00028,0.00044,0.0,0.00016,0.00028,0,0,0.00691
1,2016-01-03 22:05:00,335,0.68347,0.6836,0.68315,0.68334,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,0.68334,0.68334,...,0,0,-0.00013,0.00045,0.00013,0.00019,0.00013,5,0,NaN
2,2016-01-03 22:10:00,106,0.68348,0.68366,0.68333,0.68354,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,0.68354,0.68354,...,0,0,0.00006,0.00033,0.00012,0.00015,0.00006,10,0,NaN
3,2016-01-03 22:15:00,52,0.68365,0.68374,0.6835,0.68352,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,0.68352,0.68352,...,0,0,-0.00013,0.00024,0.00009,0.00002,0.00013,15,0,NaN
4,2016-01-03 22:20:00,77,0.68367,0.68368,0.68336,0.68363,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,0.68363,0.68363,...,0,0,-0.00004,0.00032,0.00001,0.00027,0.00004,20,0,NaN


In [27]:
df2[['utc_timestamp','delta']].rename(columns = {'utc_timestamp':'utc_timestamp','delta':'delta4'})

,utc_timestamp,delta4
0,2016-01-03 22:00:00+00:00,0.00691
1,2016-01-04 02:00:00+00:00,0.0052
2,2016-01-04 06:00:00+00:00,0.0008
3,2016-01-04 10:00:00+00:00,0.00332
4,2016-01-04 14:00:00+00:00,0.00098
...,...,...
10667,2022-11-04 01:00:00+00:00,0.00252
10668,2022-11-04 05:00:00+00:00,0.00146
10669,2022-11-04 09:00:00+00:00,0.0003
10670,2022-11-04 13:00:00+00:00,0.00028


In [66]:
df[df['minute_col'] == 55].shape[0] * 20

818560

In [67]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col,hour_col,delta4,support_indicator
0,2016-01-03 22:00:00,10,0.68374,0.68374,0.6833,0.68346,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,0.68346,0.68346,...,0,-0.00028,0.00044,0.0,0.00016,0.00028,0,22,0.00691,0
1,2016-01-03 22:05:00,335,0.68347,0.6836,0.68315,0.68334,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,0.68334,0.68334,...,0,-0.00013,0.00045,0.00013,0.00019,0.00013,5,22,-100.0,0
2,2016-01-03 22:10:00,106,0.68348,0.68366,0.68333,0.68354,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,0.68354,0.68354,...,0,0.00006,0.00033,0.00012,0.00015,0.00006,10,22,-100.0,0
3,2016-01-03 22:15:00,52,0.68365,0.68374,0.6835,0.68352,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,0.68352,0.68352,...,0,-0.00013,0.00024,0.00009,0.00002,0.00013,15,22,-100.0,0
4,2016-01-03 22:20:00,77,0.68367,0.68368,0.68336,0.68363,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,0.68363,0.68363,...,0,-0.00004,0.00032,0.00001,0.00027,0.00004,20,22,-100.0,0


In [68]:
df.columns

Index(['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
       'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
       'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
       'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
       'lower_wick', 'delta', 'minute_col', 'hour_col', 'delta4',
       'support_indicator'],
      dtype='object')

In [72]:
df[df['london'] == 1].shape

(184369, 29)

In [70]:
df[df['support_indicator'] == 1].shape

(114, 29)

In [ ]:
ml_features = ['next_eight_hours_bullish',
    'new_york',
    'london',
    'support_indicator',
    'smma_5_bullish',
    'smma_5_bearish',
    'ema_5_bullish',
    'ema_5_bearish',
    'bearish_engulfing',
    'bullish_engulfing',
    'bearish_tls',
    'bullish_tls',
    'bullish_5',
    'bullish_hour',
    'bullish_4hour',
    'bullish_8hour',
    'bullish_24hour',
    'bullish_48hour',
    'bullish_120hour',
    'bullish_250hour',
    'delta_5',
    'delta_hour',
    'delta_4hour',
    'delta_8hour',
    'delta_24hour',
    'delta_48hour',
    'delta_120hour',
    'delta_250hour',
    'upper_wick_5',
    'upper_wick_hour',
    'upper_wick_4hour',
    'lower_wick_5',
    'lower_wick_hour',
    'lower_wick_4hour',    
    
]

In [21]:
count_df = pd.DataFrame(df['utc_timestamp'].value_counts())
count_df[count_df['utc_timestamp'] > 1]

,utc_timestamp


In [22]:
df2 = df2.drop_duplicates(subset=['utc_timestamp'])
df2.shape

(10664, 14)

In [53]:


def get_support(df,
                df2,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0,
                percentile = 98
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    min_filter = np.percentile(df2['delta'],percentile)
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= min_filter and arr[i,minute_col] == 55 and sum(arr[i - 50: i,new_col]) == 0:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SUPPORT INDICATOR SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T



all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 




final_df = pd.DataFrame([0])
final_med = pd.DataFrame([0])
final_mean = pd.DataFrame([0])
final_sum = pd.DataFrame([0])
count = 0
for pair in all_pairs:
    print(pair)  

    if 'JPY' in pair:
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
        spread = .01
        high_spread = .0005 * 100
        low_spread = 0 * 100
        pip_delta = .005 * 100
    else:
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
        spread = .0001
        high_spread = .0005
        low_spread = 0
        pip_delta = .005 

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = get_support(df,
                     df2,
                lookup_range = 2,
                pip_delta = pip_delta,
                 time_range = 47,
                high_spread = high_spread,
                low_spread = low_spread,
                percentile = 99
               )
    count += df[df['support_indicator'] == 1].shape[0]
    meds,means,sums = test_future(df,pair)
    for j in range(meds.shape[1]):
        if abs(meds.iloc[0,j]) < .01:
            meds.iloc[0,j] = 0
            
    for j in range(means.shape[1]):
        if abs(means.iloc[0,j]) < .01:
            means.iloc[0,j] = 0  
            
    for j in range(sums.shape[1]):
        if abs(sums.iloc[0,j]) < .01:
            sums.iloc[0,j] = 0  
            
    if final_med.shape[1] == 1:
        final_med = meds
        final_mean = means
        final_sum = sums
    else:
        final_med = pd.concat([final_med,meds])
        final_mean = pd.concat([final_mean,means])
        final_sum = pd.concat([final_sum,sums])
print('FINAL TRADE COUNT',count)
final_med


GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 27)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_H4/2016-01-01_2022-07-31.csv
(10658, 15)
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 2 0.5
COUNT: 0
TOTAL FUNCTION TIME: 0.007320082187652588  MINUTES
SUPPORT INDICATOR SHAPE (100, 28)
MEDIAN:
next_30             4.60
next_hour          -1.75
next_two_hours      9.35
next_four_hours    -0.25
next_eight_hours    3.05
dtype: float64

MEAN:
next_30              1.429
next_hour            6.441
next_two_hours      10.478
next_four_hours     -8.727
next_eight_hours   -10.849
dtype: float64

SUM:
   next_30  next_hour  next_two_hours  next_four_hours  next_eight_hours
0    142.9      644.1          1047.8           -872.7           -1084.9
None
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 27)
PATH: /Users/aidan

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,4.60,-1.75,9.35,-0.25,3.05
EUR_USD,2.80,3.15,5.35,-1.40,3.85
USD_JPY,3.20,2.00,3.60,2.80,2.60
EUR_JPY,1.30,3.00,6.50,14.40,8.75
AUD_JPY,5.20,2.70,6.90,8.20,8.40
CAD_JPY,2.20,3.00,4.55,5.60,5.75
CHF_JPY,4.25,4.85,6.90,4.65,9.80
GBP_USD,2.40,5.20,4.55,2.95,10.20
USD_CAD,0.00,0.80,5.00,6.00,4.00
USD_CHF,0.55,0.45,1.70,1.90,9.40


In [58]:
(final_med['next_eight_hours'].mean() - 2) * count/3

1658.1365079365053

In [49]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col,hour_col,support_indicator
0,2016-01-03 22:00:00,55,86.998,87.012,86.992,86.995,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,86.995,86.995,...,0,0,-0.003,0.02,0.014,0.003,0.003,22,0,0
1,2016-01-03 22:05:00,92,86.996,86.999,86.962,86.964,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,86.964,86.964,...,0,0,-0.032,0.037,0.003,0.002,0.032,22,0,0
2,2016-01-03 22:10:00,141,86.965,86.965,86.914,86.946,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,86.946,86.946,...,0,0,-0.019,0.051,0.0,0.032,0.019,22,0,0
3,2016-01-03 22:15:00,115,86.946,86.97,86.941,86.966,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,86.966,86.966,...,0,0,0.02,0.029,0.004,0.005,0.02,22,0,0
4,2016-01-03 22:20:00,60,86.966,86.968,86.95,86.955,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,86.955,86.955,...,0,0,-0.011,0.018,0.002,0.005,0.011,22,0,0


In [50]:
df2.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col,hour_col
0,2016-01-03T22:00:00.000000000Z,12931,86.998,87.012,86.288,86.299,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.699,0.724,0.014,0.011,0.699,22,0
1,2016-01-04T02:00:00.000000000Z,18793,86.298,86.31,85.834,85.911,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,-0.387,0.476,0.012,0.077,0.387,2,0
2,2016-01-04T06:00:00.000000000Z,22289,85.91,85.95,85.186,85.488,2016-01-04 06:00:00+00:00,2016-01-04 01:00:00-05:00,-0.422,0.764,0.04,0.302,0.422,6,0
3,2016-01-04T10:00:00.000000000Z,22362,85.487,85.798,85.288,85.756,2016-01-04 10:00:00+00:00,2016-01-04 05:00:00-05:00,0.269,0.51,0.042,0.199,0.269,10,0
4,2016-01-04T14:00:00.000000000Z,31835,85.756,86.007,85.354,85.539,2016-01-04 14:00:00+00:00,2016-01-04 09:00:00-05:00,-0.217,0.653,0.251,0.185,0.217,14,0


In [12]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3

1803.8079365079302

In [38]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3

1084.7619047619057

In [59]:

def mark_four_hour(df,df2):
    df = df.merge(df2[['utc_timestamp','delta']].rename(columns = {'utc_timestamp':'utc_timestamp','delta':'delta4'}),on = 'utc_timestamp',how = 'left')
    df['delta4'] = df['delta4'].fillna(-100)
    col = df.columns.get_loc('delta4')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,col] != -100:
            arr[i - 1,col] = 1000
            
    return pd.DataFrame(arr,columns = df.columns)
def get_support(df,
                df2,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0,
                percentile = 98
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d4 = df.columns.get_loc('delta4')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    min_filter = np.percentile(df2['delta'],percentile)
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= min_filter and arr[i,minute_col] == 55 and sum(arr[i - 50: i,new_col]) == 0 and arr[i,d4] == 1000:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T



all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 




final_df = pd.DataFrame([0])
final_med = pd.DataFrame([0])
final_mean = pd.DataFrame([0])
final_sum = pd.DataFrame([0])
count = 0
for pair in all_pairs:
    print(pair)  

    if 'JPY' in pair:
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
        spread = .01
        high_spread = .0005 * 100
        low_spread = 0 * 100
        pip_delta = .005 * 100
    else:
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
        spread = .0001
        high_spread = .0005
        low_spread = 0
        pip_delta = .005 

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = mark_four_hour(df,df2)
    df = get_support(df,
                     df2,
                lookup_range = 2,
                pip_delta = pip_delta,
                 time_range = 47,
                high_spread = high_spread,
                low_spread = low_spread,
                percentile = 99
               )
    count += df[df['support_indicator'] == 1].shape[0]
    meds,means,sums = test_future(df,pair)
    for j in range(meds.shape[1]):
        if abs(meds.iloc[0,j]) < .01:
            meds.iloc[0,j] = 0
            
    for j in range(means.shape[1]):
        if abs(means.iloc[0,j]) < .01:
            means.iloc[0,j] = 0  
            
    for j in range(sums.shape[1]):
        if abs(sums.iloc[0,j]) < .01:
            sums.iloc[0,j] = 0  
            
    if final_med.shape[1] == 1:
        final_med = meds
        final_mean = means
        final_sum = sums
    else:
        final_med = pd.concat([final_med,meds])
        final_mean = pd.concat([final_mean,means])
        final_sum = pd.concat([final_sum,sums])
print('FINAL TRADE COUNT',count)
final_med


GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 27)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_H4/2016-01-01_2022-07-31.csv
(10658, 15)
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 2 0.5
COUNT: 0
TOTAL FUNCTION TIME: 0.00790996551513672  MINUTES
SHAPE (49, 29)
MEDIAN:
next_30             3.2
next_hour           3.4
next_two_hours      3.9
next_four_hours     6.6
next_eight_hours    7.5
dtype: float64

MEAN:
next_30             -1.059184
next_hour            7.567347
next_two_hours     -28.420408
next_four_hours    -31.926531
next_eight_hours   -17.618367
dtype: float64

SUM:
   next_30  next_hour  next_two_hours  next_four_hours  next_eight_hours
0    -51.9      370.8         -1392.6          -1564.4            -863.3
None
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 27)
PATH: /Users/aidanmcconnell/

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,3.20,3.40,3.90,6.60,7.50
EUR_USD,0.60,-0.15,-2.70,-5.70,2.40
USD_JPY,-3.70,-2.10,-5.80,-0.40,-3.80
EUR_JPY,-2.40,0.20,0.30,-9.80,-0.20
AUD_JPY,-1.70,3.80,5.60,8.20,11.10
CAD_JPY,-1.50,2.90,4.80,-0.70,2.70
CHF_JPY,-2.10,-0.35,-0.60,-4.95,-9.55
GBP_USD,2.25,4.80,6.60,-4.70,-7.00
USD_CAD,1.80,2.50,5.30,-1.55,5.15
USD_CHF,0.75,0.95,1.90,2.05,4.00


In [60]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3

695.9492063491967

In [ ]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3

In [36]:
(meds['next_eight_hours'].mean() - 2) * count

9540.80000000038

In [39]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3

1084.7619047619057

In [42]:
(final_sum['next_eight_hours'].sum()) / 3

1529.7666666666737

In [61]:

def mark_four_hour(df,df2):
    df = df.merge(df2[['utc_timestamp','delta']].rename(columns = {'utc_timestamp':'utc_timestamp','delta':'delta4'}),on = 'utc_timestamp',how = 'left')
    df['delta4'] = df['delta4'].fillna(-100)
    col = df.columns.get_loc('delta4')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,col] != -100:
            arr[i - 1,col] = 1000
            
    return pd.DataFrame(arr,columns = df.columns)
def get_support(df,
                df2,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0,
                percentile = 98
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    d4 = df.columns.get_loc('delta4')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    min_filter = np.percentile(df2['delta'],percentile)
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= min_filter and arr[i,minute_col] == 55 and sum(arr[i - 50: i,new_col]) == 0 and arr[i,d4] == -100:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def test_future2(df,hour,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    hour_col = df.columns.get_loc('hour_col')
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1 and arr[i,hour_col] == hour:
            c = arr[i,c_col]

            lst.append([hour,mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['hour','next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T

all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 




final_df = pd.DataFrame([0])
final_med = pd.DataFrame([0])
final_mean = pd.DataFrame([0])
final_sum = pd.DataFrame([0])
count = 0
for pair in all_pairs:
    print(pair)  

    if 'JPY' in pair:
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
        spread = .01
        high_spread = .0005 * 100
        low_spread = 0 * 100
        pip_delta = .005 * 100
    else:
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
        spread = .0001
        high_spread = .0005
        low_spread = 0
        pip_delta = .005 

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = mark_four_hour(df,df2)
    df = get_support(df,
                     df2,
                lookup_range = 2,
                pip_delta = pip_delta,
                 time_range = 47,
                high_spread = high_spread,
                low_spread = low_spread,
                percentile = 99
               )
    count += df[df['support_indicator'] == 1].shape[0]
    meds,means,sums = test_future(df,pair)
    for j in range(meds.shape[1]):
        if abs(meds.iloc[0,j]) < .01:
            meds.iloc[0,j] = 0
            
    for j in range(means.shape[1]):
        if abs(means.iloc[0,j]) < .01:
            means.iloc[0,j] = 0  
            
    for j in range(sums.shape[1]):
        if abs(sums.iloc[0,j]) < .01:
            sums.iloc[0,j] = 0  
            
    if final_med.shape[1] == 1:
        final_med = meds
        final_mean = means
        final_sum = sums
    else:
        final_med = pd.concat([final_med,meds])
        final_mean = pd.concat([final_mean,means])
        final_sum = pd.concat([final_sum,sums])
        
print('FINAL TRADE COUNT',count)
final_med



GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 27)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_H4/2016-01-01_2022-07-31.csv
(10658, 15)
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 2 0.5
COUNT: 0
TOTAL FUNCTION TIME: 0.007863910992940266  MINUTES
SHAPE (90, 29)
MEDIAN:
next_30             5.20
next_hour          -3.10
next_two_hours      7.90
next_four_hours    -3.00
next_eight_hours   -0.95
dtype: float64

MEAN:
next_30              3.112222
next_hour            1.350000
next_two_hours       7.846667
next_four_hours    -17.861111
next_eight_hours   -17.845556
dtype: float64

SUM:
   next_30  next_hour  next_two_hours  next_four_hours  next_eight_hours
0    280.1      121.5           706.2          -1607.5           -1606.1
None
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 27)
PATH: /Users/aidanmcco

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,5.20,-3.10,7.90,-3.00,-0.95
EUR_USD,3.20,4.00,6.10,0.20,3.45
USD_JPY,3.45,4.75,3.30,2.00,5.20
EUR_JPY,-0.10,2.15,6.30,14.40,8.75
AUD_JPY,5.20,2.20,2.80,6.50,4.70
CAD_JPY,2.60,1.40,2.00,5.60,5.90
CHF_JPY,3.95,2.45,5.45,6.30,8.70
GBP_USD,2.60,4.50,1.80,6.80,10.30
USD_CAD,0.00,0.80,5.00,2.90,-4.00
USD_CHF,0.50,-0.30,1.60,0.70,10.60


In [63]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3



813.6158730156641

In [57]:
((final_med['next_eight_hours'].mean() - 2) * count) / 3

1658.1365079365053

In [46]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col,hour_col,delta4,support_indicator
0,2016-01-03 22:00:00,10,0.68374,0.68374,0.6833,0.68346,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,0.68346,0.68346,...,0,-0.00028,0.00044,0.0,0.00016,0.00028,0,0,0.00691,0
1,2016-01-03 22:05:00,335,0.68347,0.6836,0.68315,0.68334,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,0.68334,0.68334,...,0,-0.00013,0.00045,0.00013,0.00019,0.00013,5,0,-100.0,0
2,2016-01-03 22:10:00,106,0.68348,0.68366,0.68333,0.68354,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,0.68354,0.68354,...,0,0.00006,0.00033,0.00012,0.00015,0.00006,10,0,-100.0,0
3,2016-01-03 22:15:00,52,0.68365,0.68374,0.6835,0.68352,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,0.68352,0.68352,...,0,-0.00013,0.00024,0.00009,0.00002,0.00013,15,0,-100.0,0
4,2016-01-03 22:20:00,77,0.68367,0.68368,0.68336,0.68363,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,0.68363,0.68363,...,0,-0.00004,0.00032,0.00001,0.00027,0.00004,20,0,-100.0,0
